# Check the Validity of Simulated Annealing (SA)

In [1]:
from functions_SFMA import *

## Load W matrices and mean and standard deviation (std) values 

In [2]:
num_N = 10 # number of rows of a matrix 

In [3]:
n_bit = num_N*num_K # number of spins (qubits)
val_dim = f'd{n_bit}'

# Load W matrices and mean and std values
folder0 =  '20241227_pFMA_Wmatrix/results/z900/' 
name0   =  f'C6_{num_N}x2'
init_file = f'{folder0}/{name0}.pickle.bz2'
with bz2.open(init_file, 'rb') as f:
        param = pickle.load(f)

# Load the W matrices
list_Wmatrix = [] # list for collecting the W matrices 
for w in param['tdata']:
     list_Wmatrix.append(w) # append the W matrix to list_Wmatrix

# Load the mean and std values
obj_mean = param['obj_func_mean'] # array of the mean values of the objective functions (output values)
obj_std = param['obj_func_std']   # array of the std values of the outputs

## Create inital datasets, train non-standardized and standardized Factorization Machine (FM) models using them, and find the optimal solutions via SA. Further, perform exhaustive (brute force) search methods.   

In [4]:
#################### Parameters for initial datasets and FM models
n_in = n_bit # size of initial datasets 
k_FM = int(n_bit/2)-1 # hyperparameter for FM models 

############################### Calculate the optimal solutions by both exhaustive search and SA.
all_binaries = all_binaries(n_bit) # generate all binary configurations
list_sort_binary_indices_nonstandard = [] # list for collecting sorted indices of binary solutions for ten W matrices (non-standardized FM models)
list_sort_binary_indices_standard = [] # list for collecting sorted indices of binary solutions for ten W matrices (standardized FM models)
list_xhat_nonstandard = [] # list of the best candidate solutions for ten W matrices (non-standardized FM models)
list_xhat_standard = [] # list of the best candidate solutions for ten W matrices (standardized FM models)
for num_W in range(d_W): # perform SA and an exhaustive search method for each W matrix
    val_num_W = f'W_{num_W}' 
    Matrix_W = list_Wmatrix[num_W] # W matrix
    mean = obj_mean[num_W] # mean
    std = obj_std[num_W] # std 
    xin, yin = init_training_data(Matrix_W, num_N, num_K, n_bit, n_in, nin_seed) #  generating an inital dataset
    yin_nonstandard = yin # non-standardized output value
    yin_standard = standard(yin, mean, std) # standardized output value
    ################# Train non-standardized FM models.
    torch.manual_seed(seed_intial_FMTorch) # set a random seed for reproducibility 
    model_FM_nonstandard = TorchFM(d=n_bit, k=k_FM, Stand_Dev=1) # instantiate a FM model 
    trained_FMmodel_nonstandard = train_FM_model(xin, yin_nonstandard, model_FM_nonstandard) # train a FM model
    _, trained_parameters_nonstandard = trained_FMmodel_nonstandard # get FM model parameters (v,w,w0) 
    v_nonstandard, w_nonstandard, w0_nonstandard = trained_parameters_nonstandard # get v,w, and w0  
    M_QUBO_nonstandard = construct_QUBO(w_nonstandard, v_nonstandard) # construct a QUBO matrix using w and v
    list_QUBO_nonstandard = [] # list of the values of a FM function for all binaries
    for x in all_binaries:
        y = calculate_binary_quadratic(x, M_QUBO_nonstandard, w0_nonstandard) # value of the FM function
        list_QUBO_nonstandard.append(y) # append y to list_QUBO_nonstandard 
    array_QUBO_nonstandard = np.array(list_QUBO_nonstandard) # convert list_QUBO_nonstandard to a NumPy array
    ## exhaustive search
    sorted_binary_indices_QUBO_nonstandard =  np.argsort(array_QUBO_nonstandard) # sorted indices of the output values
    list_sort_binary_indices_nonstandard.append(sorted_binary_indices_QUBO_nonstandard) # for each W matrix, append the sorted indices 
    ################## SA
    response_x_nonstandard, response_y_nonstandard, xstar_nonstandard, ystar_nonstandard = simulated_anneal(trained_FMmodel_nonstandard, 
                                         Matrix_W, num_N, num_K) # run SA
    
    # response_x_nonstandard: array of nreads_SA candiate solutions
    # response_y_nonstandard: array of nreads_SA output values of the candiate solutions
    # xstar_nonstandard: best candiate solution
    # ystar_nonstandard: output value of the best candiate solution: the minimum value
    list_xhat_nonstandard.append(xstar_nonstandard) # for each W matrix, append xstar_nonstandard to list_xhat_nonstandard 
    
    ################# Train standardized FM models.
    torch.manual_seed(seed_intial_FMTorch) # set a random seed for reproducibility 
    model_FM_standard = TorchFM(d=n_bit, k=k_FM, Stand_Dev=std) # instantiate a FM model 
    trained_FMmodel_standard = train_FM_model(xin, yin_standard, model_FM_standard) # train a FM model
    _, trained_parameters_standard = trained_FMmodel_standard # get FM model parameters (v,w,w0) 
    v_standard, w_standard, w0_standard = trained_parameters_standard # get v,w, and w0  
    M_QUBO_standard = construct_QUBO(w_standard, v_standard) # construct a QUBO matrix using w and v
    list_QUBO_standard = [] # list of the values of a FM function for all binaries
    for x in all_binaries:
        y = calculate_binary_quadratic(x, M_QUBO_standard, w0_standard) # value of the FM function
        list_QUBO_standard.append(y) # append y to list_QUBO_standard 
    array_QUBO_standard = np.array(list_QUBO_standard) # convert list_QUBO_standard into a NumPy array
    ## exhaustive search
    sorted_binary_indices_QUBO_standard =  np.argsort(array_QUBO_standard) # sorted indices of the output values
    list_sort_binary_indices_standard.append(sorted_binary_indices_QUBO_standard) # for each W matrix, append the sorted indices 
    ################## SA
    response_x_standard, response_y_standard, xstar_standard, ystar_standard = simulated_anneal(trained_FMmodel_standard, 
                                         Matrix_W, num_N, num_K) # run SA
    
    # response_x_standard: array of nreads_SA candiate solutions
    # response_y_standard: array of nreads_SA output values of the candiate solutions
    # xstar_standard: best candiate solution
    # ystar_standard: the minimum value
    
    list_xhat_standard.append(xstar_standard) # for each W matrix, append xstar_standard to list_xhat_standard 


## Compare the best candidate (optimal) solutions obtained by SA with that achieved by the exhaustive search methods

In [5]:
print('non-standard')
for num_W in range(d_W ): 
    val_num_W = f'W_{num_W}' 
    sorted_indices_nonstandard = list_sort_binary_indices_nonstandard[num_W] # sorted indices obtained by exhaustive search
    x_neal = list_xhat_nonstandard[num_W] # best candidate solution obtained by SA
    index_x_neal = binary_array_to_decimal(x_neal) # index of the best candidate solution 
    print(val_num_W,  "neal:", index_x_neal, "exaustive:", sorted_indices_nonstandard[0])

non-standard
W_0 neal: 523711 exaustive: 523711
W_1 neal: 523711 exaustive: 523711
W_2 neal: 523711 exaustive: 523711
W_3 neal: 523711 exaustive: 523711
W_4 neal: 523711 exaustive: 523711
W_5 neal: 523711 exaustive: 523711
W_6 neal: 523711 exaustive: 523711
W_7 neal: 523711 exaustive: 523711
W_8 neal: 523711 exaustive: 523711
W_9 neal: 523711 exaustive: 523711


In [6]:
print('standard')
for num_W in range(d_W ): 
    val_num_W = f'W_{num_W}' 
    sorted_indices_standard = list_sort_binary_indices_standard[num_W] # sorted indices obtained by exhaustive search
    x_neal = list_xhat_standard[num_W] # best candidate solution obtained by SA
    index_x_neal = binary_array_to_decimal(x_neal) # index of the best candidate solution 
    print(val_num_W,  "neal:", index_x_neal, "exaustive:", sorted_indices_standard[0])

standard
W_0 neal: 979645 exaustive: 979645
W_1 neal: 995254 exaustive: 995254
W_2 neal: 892188 exaustive: 892188
W_3 neal: 747452 exaustive: 747452
W_4 neal: 964267 exaustive: 964267
W_5 neal: 1038662 exaustive: 1038662
W_6 neal: 457525 exaustive: 457525
W_7 neal: 454637 exaustive: 454637
W_8 neal: 778683 exaustive: 778683
W_9 neal: 486129 exaustive: 486129
